# SQL Parte 2:

## Objetivos:
    1. Organización de tablas.
    2. Introducción a joins entre tablas.
    



In [9]:
import pandas as pd
import duckdb

In [10]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [11]:
duck_query = lambda q: duckdb.query(q)

In [21]:
duck_query(
    """
    INSTALL mysql;
    LOAD mysql;
    """
)

Utilizaremos la siguiente conexión: https://relational.fel.cvut.cz/dataset/IMDb

<img src="IMDB_Schema.png" />

In [ ]:
duck_query(
    """
    ATTACH 'host=relational.fel.cvut.cz user=guest password=ctu-relational port=3306 database=imdb_ijs' AS imdb (TYPE MYSQL);
    """
)

In [34]:
duck_query(
    """
    SHOW TABLES
    """
)

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

### 2. Enviar un query a la base de datos y obtenerlo como un dataframe de Pandas:

    Utilizaremos la función creada anteriormente "duck_query" para obtener los resultados de nuestros querys: 

In [37]:
duck_query(
    """
    SELECT *
    FROM
        imdb.actors
    LIMIT 5
    """
).to_df()

,id,first_name,last_name,gender
0,2,Michael,'babeepower' Viera,M
1,3,Eloy,'Chincheta',M
2,4,Dieguito,'El Cigala',M
3,5,Antonio,'El de Chipiona',M
4,6,José,'El Francés',M


### 3. Organización de tablas

### 3.1 Crear y eliminar tablas

In [38]:
duck_query(
    """
    CREATE OR REPLACE TABLE jjosue_temp_actors AS
    SELECT *
    FROM
        imdb.actors
    WHERE
        id BETWEEN 1 AND 99
    ;
    """
)

In [39]:
duck_query(
    """
    DROP TABLE jjosue_temp_actors;
    """
)

Otras formas de trabajar con creacion de tablas sería con comandos como:

* DROP TABLE jjosue_temp_actors;
* CREATE TABLE jjosue_temp_actors AS ...;

Sin embargo, se recomienda utilizar REPLACE para no tener problemas al momento de eliminar una tabla que aún no ha sido creada.

<img src="Create_Table.png" />
<img src="Create_table_syntax.png" />

In [40]:
list_df = duck_query(
    """
    SHOW ALL TABLES 
    """
).to_df()

In [47]:
list_df[list_df['database']=='imdb']

,database,schema,name,column_names,column_types,temporary
0,imdb,Accidents,nesreca,"[id_nesreca, klas_nesreca, upravna_enota, cas_...","[VARCHAR, VARCHAR, VARCHAR, TIMESTAMP, VARCHAR...",False
1,imdb,Accidents,oseba,"[id_nesreca, povzrocitelj_ali_udelezenec, star...","[VARCHAR, VARCHAR, UTINYINT, VARCHAR, VARCHAR,...",False
2,imdb,Accidents,upravna_enota,"[id_upravna_enota, ime_upravna_enota, st_prebi...","[VARCHAR, VARCHAR, UINTEGER, USMALLINT]",False
3,imdb,Ad,ad,"[ad_id, user_id, ts]","[INTEGER, INTEGER, TIMESTAMP WITH TIME ZONE]",False
4,imdb,AdventureWorks2014,AWBuildVersion,"[SystemInformationID, Database Version, Versio...","[UTINYINT, VARCHAR, TIMESTAMP, TIMESTAMP WITH ...",False
...,...,...,...,...,...,...
1840,imdb,voc,total,"[number, number_sup, trip, trip_sup, onboard_a...","[INTEGER, VARCHAR, INTEGER, VARCHAR, INTEGER, ...",False
1841,imdb,voc,voyages,"[artificial_id, number, number_sup, trip, trip...","[VARCHAR, INTEGER, VARCHAR, INTEGER, VARCHAR, ...",False
1842,imdb,world,City,"[ID, Name, CountryCode, District, Population]","[INTEGER, VARCHAR, VARCHAR, VARCHAR, INTEGER]",False
1843,imdb,world,Country,"[Code, Name, Continent, Region, SurfaceArea, I...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, FLOAT, SM...",False


### 3.2 Insertar y eliminar registros

In [48]:
duck_query(
    """
    SELECT COUNT(*) FROM jjosue_temp_actors;
    """
)

CatalogException: Catalog Error: Table with name jjosue_temp_actors does not exist!
Did you mean "imdb.tpce.E_SECTOR"?

In [ ]:
duck_query(
    """
    INSERT INTO jjosue_temp_actors
    SELECT *
    FROM
        imdb.actors
    WHERE 
        id BETWEEN 100 AND 199
    """
)

In [ ]:
duck_query(
    """
    SELECT COUNT(*) FROM jjosue_temp_actors;
    """
)

In [ ]:
duck_query(
    """
    DELETE FROM jjosue_temp_actors WHERE id BETWEEN 46 AND 56;
    """
)

In [ ]:
duck_query(
    """
    SELECT COUNT(*) FROM jjosue_temp_actors;
    """
)

### 4. Introducción a Joins entre tablas:

#### Teoría de conjuntos: 
    Conjunto: 
        -Una colección de distintos elementos (Tsitsiklis).
        -Una agrupación, colección o reunión de objeto que cumplen una propiedad determinada.
    
    A los objetos del conjunto se denominan elementos. 
    
    
#### Ejemplos:
    1. El conjunto de estudiantes de data wrangling.
    2. El conjunto de letras de "data wrangling". 
    
#### Notación: 
    Los conjuntos se denotan con letras mayúsculas del alfabeto. Los elementos se escriben dentro de llaves.
    
    A = El conjunto de estudiantes de data wrangling. 
    B = El conjunto de letras de "data wrangling".
    
#### Determinación de conjuntos: 
    Extensión: se enumeran o se nombran los elementos del conjunto.
        B = {d,a,t,w,r,n,g,l} 
    Comprensión: Se enuncia la propiedad o la cualidad que distingue a los elementos.
        B = {x/ x es una letra de las palabras "data wrangling"}
    

#### Conjuntos especiales

        Conjunto vacío: aquel en el que ningún elemento cumple con la propiedad conocida como la regla de elegibilidad. 
        (Para cada familia de conjuntos, existe otro conjunto que contiene un elemento de cada uno de aquellos.)

    (Ver axioma de existencia) 

#### Propiedades de los conjuntos 

<img src="set_theory.png" />

- La intersección de conjuntos es conmutativa.
- La intersección de conjuntos es asociativa. 
- La intersección es distributiva respecto a la unión.
- La unión es distributiva respecto a la intersección.
- La unión de un conjunto con el universo es el universo.
- La intersección de un conjunto con su complemento es el conjunto vacío.



#### Joins entre tablas. 

Una de las aplicaciones más palpables de teoría de conjuntos en la ciencia de datos (aparte de teoría de probabilidad) es en manipulación y union entre sets de datos. De cierta manera, podemos pensar en nuestras colecciones de datos como conjuntos de elementos que comparten ciertas características en común. Sin embargo, estas características están dispersas en varias fuentes de información y parte previa al análisis es el proceso de consolidación, por lo cual se vuelve necesario realizar operaciones de conjuntos con ellos. 

Para esto, utilizaremos las keywords JOIN de SQL.


<img src="joins.jpg" />




### Ejemplo 1: Encontrar el ID del director de las películas de Star Wars

In [99]:
duck_query(
    """
    CREATE OR REPLACE TEMPORARY TABLE star_wars_movies AS
    SELECT DISTINCT
        a.id
        ,a.name
        ,a.year
        ,b.director_id
    FROM
        imdb.movies AS a
    LEFT JOIN
        imdb.movies_directors AS b
        ON a.id = b.movie_id
    WHERE
        LOWER(TRIM(name)) LIKE '%star wars%'
    ORDER BY year DESC
    LIMIT 10
    ;
    """
)

In [100]:
duck_query(
    """
    SELECT * 
    FROM star_wars_movies
    ;
    """
)

┌────────┬──────────────────────────────────────────────────────┬───────┬─────────────┐
│   id   │                         name                         │ year  │ director_id │
│ int32  │                       varchar                        │ int32 │    int32    │
├────────┼──────────────────────────────────────────────────────┼───────┼─────────────┤
│ 313477 │ Star Wars: Episode III - Revenge of the Sith         │  2005 │       48115 │
│ 313496 │ Star Wars: Republic Commando                         │  2005 │        NULL │
│ 289116 │ Saving Star Wars                                     │  2004 │       86465 │
│ 313466 │ Star Wars: Battlefront                               │  2004 │        NULL │
│ 100121 │ Empire of Dreams: The Story of the Star Wars Trilogy │  2004 │       10809 │
│ 100121 │ Empire of Dreams: The Story of the Star Wars Trilogy │  2004 │        5662 │
│ 313465 │ Star Wars Rogue Squadron III: Rebel Strike           │  2003 │       22349 │
│ 406411 │ "Star Wars: Clone War

### Ejercicio 1: 

Obtener las películas de Steven Spielberg y ordenar por año de manera descendiente.

### Ejemplo 2: Actores y roles de películas

In [101]:
duck_query(
    """    
    SELECT
        a.*
        ,b.role
        ,c.first_name || ' ' || c.last_name AS actor_name
    FROM
        star_wars_movies AS a
    LEFT JOIN
        imdb.roles AS b
        ON a.id = b.movie_id
    LEFT JOIN
        imdb.actors AS c
        ON b.actor_id = c.id
    WHERE
        a.name like 'Star Wars: Episode III%'
    --ORDER BY a.id, a.name, a.year DESC
    ;
    """
)

┌────────┬─────────────────────────────┬───────┬─────────────┬───────────────────────────────┬─────────────────────────┐
│   id   │            name             │ year  │ director_id │             role              │       actor_name        │
│ int32  │           varchar           │ int32 │    int32    │            varchar            │         varchar         │
├────────┼─────────────────────────────┼───────┼─────────────┼───────────────────────────────┼─────────────────────────┤
│ 313477 │ Star Wars: Episode III - …  │  2005 │       48115 │ Motee                         │ Kristy Wright           │
│ 313477 │ Star Wars: Episode III - …  │  2005 │       48115 │ Ryoo Naberrie                 │ Keira Wingate           │
│ 313477 │ Star Wars: Episode III - …  │  2005 │       48115 │ A Jedi Knight                 │ Aliyah Williams         │
│ 313477 │ Star Wars: Episode III - …  │  2005 │       48115 │ Senator Terr Taneel           │ Motshabi Tyelele        │
│ 313477 │ Star Wars: Episode II

### Ejercicio 2: 

2.1 Cuántos directores han dirigido las películas de Harry Potter?

2.2 Cuáles son los géneros más comunes de los directores de las películas de Harry Potter?